### TRAIN

In [1]:
import pickle
import spacy
import random
nlp = spacy.blank('en')

#train data
train_data= pickle.load(open('/Users/cosmos/Desktop/DeepBlue/BACKEND/ML/Datasets (UNCLEANED)/train_data_700.pkl','rb'))
#train_data= pickle.load(open('/content/drive/MyDrive/Colab Notebooks/datasets/d.pkl','rb'))

#defining the traning model

def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner=nlp.create_pipe('ner')
        nlp.add_pipe(ner,last=True)
        
    for _, annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])

# Now to remove other pipelines 

    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
            optimizer = nlp.begin_training()
            for itn in range(50):
                print("Statring iteration " + str(itn))
                random.shuffle(train_data)
                losses = {}
                index = 0
                for text, annotations in train_data:
                    try:
                        nlp.update(
                            [text],  # batch of texts
                            [annotations],  # batch of annotations
                            drop=0.2,  # dropout - make it harder to memorise data
                            sgd=optimizer,  # callable to update weights
                            losses=losses)
                    except Exception as e:
                        pass
                    
                print(losses)

Init Plugin
Init Graph Optimizer
Init Kernel


In [ ]:
train_model(train_data) #7159

In [ ]:
nlp.to_disk('/content/drive/MyDrive/Colab Notebooks/Models/DeepBlue/700-new-code')

### PREDICT

import textract
import re
from tika import parser 
fname = '/content/drive/MyDrive/Colab Notebooks/Models/DeepBlue/Test- Resumes/Resumes/Resume 2 pdf/type_14.pdf'

# text1 = textract.process('/content/drive/MyDrive/Colab Notebooks/Models/DeepBlue/Test- Resumes/Profile.pdf')
# text2 = textract.process('/content/drive/MyDrive/Colab Notebooks/Models/DeepBlue/Test- Resumes/test.pdf')

raw = parser.from_file(fname)
print(raw['content'])


def pre_process(text):
  text = re.sub(r'[^\x00-\x7f]',r'', text)
  text = re.sub("\n", " ",text)
  return text

#pre_process(text2) 

In [ ]:
import pickle
train_data= pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Models/DeepBlue/spacy/train_data.pkl','rb')) #trying model 2 bigger dataset


doc = nlp_model(raw['content'])
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}} - {ent.text}')

### EXTRA